# Editing Phosphosite Data Tables in Python

Three phosphosite data tables have been downloaded as tab-delimited files from https://www.phosphosite.org/staticDownloads:

- Phosphorylation_site_dataset
    - The full list of phosphosites available on the website (371,203 rows long on 2020_01_11)
  
- Kinase_Substrate_Dataset
    - A list of known kinase-phosphosite relationships (18,455 rows long on 2020_01_11)
    
- Disease-associated_sites
    - A list of phosphosites with known links to disease (1,363 rows on on 2020_01_11)
   
They will be imported as pandas dataframes, and will have the following changes made to them, in this Jupyter notebook:
   - Remove any rows with information about non-human proteins (targets and kinases).
   - Remove any rows where "gene" is blank (this field is essential for creating phosphosite and kinase IDs that will enable different tables to be linked).
   - Remove any rows in the diseases table where "disease" is blank.
   - For tables with a "modified residue" column that includes a flag at the end to denote the type of post-translational modification (PTM), remove any that are not "-p" for "phosphorylation".
   - Generate "phosphosite ID" columns, using information in other columns, to match the format in user-submitted quantitative phosphoproteomics files, and to allow the tables to be linked in the SQL database.
   - Add rows to the phosphosites table for any missing phosphosites that appear in the other tables, to allow them to be linked.
   - For the kinase-substrate table, generate a new column for the kinase name that will match the kinase table in the SQL database.
   - Ensure all genes listed are in uppercase.
   - Remove the unnecessary "-p" from the "modified residue" column in the phosphosite table, as our web app will only specialise in phosphosites.
   - Drop any other unnecessary columns.
   - Add a binary column to the phosphosites table to indicate whether the row is for an isoform (1) or not (0).
   - In the phosphosites table, remove the very few duplicated rows (by phosphosite, where the duplication is not due to the presence of isoforms)
   - Add a primary keys column to each table.

Finally, they will be exported as .csv files.

Import required packages

In [1]:
import pandas as pd
import re
import urllib.parse
import urllib.request

Read in data files

In [2]:
phosphosite_df = pd.read_table("Phosphorylation_site_dataset", error_bad_lines = False)

In [3]:
kin_sub_df = pd.read_table("Kinase_Substrate_Dataset", error_bad_lines = False)

In [4]:
disease_df = pd.read_table("Disease-associated_sites", error_bad_lines = False)

# A strange column "Unnamed: 19" is created upon import: drop this
disease_df = disease_df.drop(['Unnamed: 19'], axis=1)

Remove any rows where ORGANISM is not human

In [5]:
phosphosite_df = phosphosite_df.drop(phosphosite_df[phosphosite_df.ORGANISM != "human"].index)

In [6]:
# kin_sub_df contains a substrate / target / phosphosite organism and a kinase organism

kin_sub_df = kin_sub_df.drop(kin_sub_df[kin_sub_df.SUB_ORGANISM != "human"].index)
kin_sub_df = kin_sub_df.drop(kin_sub_df[kin_sub_df.KIN_ORGANISM != "human"].index)

In [7]:
disease_df = disease_df.drop(disease_df[disease_df.ORGANISM != "human"].index)

Remove any rows where GENE is blank

In [8]:
phosphosite_df = phosphosite_df.dropna(subset=["GENE"])
phosphosite_df = phosphosite_df.reset_index(drop=True)

In [9]:
# kin_sub_df has a kinase gene and a substrate gene

kin_sub_df = kin_sub_df.dropna(subset=["GENE"])
kin_sub_df = kin_sub_df.dropna(subset=["SUB_GENE"])
kin_sub_df = kin_sub_df.reset_index(drop=True)

In [10]:
disease_df = disease_df.dropna(subset=["GENE"])
disease_df = disease_df.reset_index(drop=True)

Remove any rows where DISEASE is blank

In [11]:
disease_df = disease_df.dropna(subset=["DISEASE"])
disease_df = disease_df.reset_index(drop=True)

Remove any rows where PTM is not "phosphorylation" (indicated as "-p" appended to MOD_RSD). N.B. kin_sub_df is not included here because its MOD_RSD column does not contain modification type, just AA and residue no.

In [12]:
# This takes around five minutes to run, and should not be necessary as
# the file just contains phosphosites, but is included in case of error

regex = r'\w{1}\d+-p'

indices = []

for n,i in enumerate(phosphosite_df.iterrows()):
    if re.findall(regex,str(i[1])):
        continue
    else:
        indices.append(n)

phosphosite_df = phosphosite_df.drop(indices)
phosphosite_df = phosphosite_df.reset_index(drop=True)

In [13]:
regex = r'\w{1}\d+-p'

indices = []

for n,i in enumerate(disease_df.iterrows()):
    if re.findall(regex,str(i[1])):
        continue
    else:
        indices.append(n)

disease_df = disease_df.drop(indices)
disease_df = disease_df.reset_index(drop=True)

Make a column of phosphosite IDs in a standardised format:
- to enable tables to be linked in the SQL database
- to match the format in the user-submitted quantitative phosphoproteomics files
- The kin_sub_df table will need three extra columns to help match as many user-submitted records as possible
- The phosphosites table should have the extra columns too, in case it is required for any phosphosites listed that aren't in kin_sub_df

In [15]:
phos_id = []
phos_id2 =[]
phos_id3 = []
phos_id4 = []

for n,i in enumerate(kin_sub_df.iterrows()):
    phos_id.append(kin_sub_df.SUB_GENE[n].upper()+"_"+kin_sub_df.SUB_ORGANISM[n].upper()+"("+kin_sub_df.SUB_MOD_RSD[n].upper()+")")
    phos_id2.append(kin_sub_df.SUBSTRATE[n].upper()+"("+kin_sub_df.SUB_MOD_RSD[n].upper()+")")
    phos_id3.append(kin_sub_df.SUBSTRATE[n].upper()+"_"+kin_sub_df.SUB_ORGANISM[n].upper()+"("+kin_sub_df.SUB_MOD_RSD[n].upper()+")")
    phos_id4.append(kin_sub_df.SUB_GENE[n].upper()+"("+kin_sub_df.SUB_MOD_RSD[n].upper()+")")
    
phos_id = pd.Series(phos_id)
phos_id2 = pd.Series(phos_id2)
phos_id3 = pd.Series(phos_id3)
phos_id4 = pd.Series(phos_id4)

kin_sub_df = kin_sub_df.assign(PHOS_ID = phos_id)
kin_sub_df = kin_sub_df.assign(PHOS_ID2 = phos_id2)
kin_sub_df = kin_sub_df.assign(PHOS_ID3 = phos_id3)
kin_sub_df = kin_sub_df.assign(PHOS_ID4 = phos_id4)

In [16]:
# This takes around three minutes to run

phos_id = []
phos_id2 =[]
phos_id3 = []
phos_id4 = []

for n,i in enumerate(phosphosite_df.iterrows()):
    phos_id.append(phosphosite_df.GENE[n].upper()+"_"+phosphosite_df.ORGANISM[n].upper()+"("+phosphosite_df.MOD_RSD[n][0:-2].upper()+")")
    phos_id2.append(phosphosite_df.PROTEIN[n].upper()+"("+phosphosite_df.MOD_RSD[n][0:-2].upper()+")")
    phos_id3.append(phosphosite_df.PROTEIN[n].upper()+"_"+phosphosite_df.ORGANISM[n].upper()+"("+phosphosite_df.MOD_RSD[n][0:-2].upper()+")")
    phos_id4.append(phosphosite_df.GENE[n].upper()+"("+phosphosite_df.MOD_RSD[n][0:-2].upper()+")")

phos_id = pd.Series(phos_id)
phos_id2 = pd.Series(phos_id2)
phos_id3 = pd.Series(phos_id3)
phos_id4 = pd.Series(phos_id4)

phosphosite_df = phosphosite_df.assign(PHOS_ID = phos_id)
phosphosite_df = phosphosite_df.assign(PHOS_ID2 = phos_id2)
phosphosite_df = phosphosite_df.assign(PHOS_ID3 = phos_id3)
phosphosite_df = phosphosite_df.assign(PHOS_ID4 = phos_id4)

In [17]:
phos_id = []

for n,i in enumerate(disease_df.iterrows()):
    phos_id.append(disease_df.GENE[n].upper()+"_"+disease_df.ORGANISM[n].upper()+"("+disease_df.MOD_RSD[n][0:-2].upper()+")")

phos_id = pd.Series(phos_id)

disease_df = disease_df.assign(PHOS_ID = phos_id)

Add rows to phosphosite_df for any missing phosphosites that appear in the other tables

In [18]:
# This takes around five minutes to run

for i in kin_sub_df['PHOS_ID']:
    if i not in list(phosphosite_df['PHOS_ID']):
        row = [{'GENE':'','PROTEIN':'','ACC_ID':'','HU_CHR_LOC':'','MOD_RSD':'','SITE_GRP_ID':0,'ORGANISM':'','MW_kD':0.00,'DOMAIN':'','SITE_+/-7_AA':'','LT_LIT':0.0,'MS_LIT':0.0,'MS_CST':0.0,'CST_CAT#':0.0,'PHOS_ID':i,'PHOS_ID2':'','PHOS_ID3':'','PHOS_ID4':''}]
        phosphosite_df = phosphosite_df.append(row, ignore_index = True)

In [19]:
# This takes minute to run

for i in disease_df['PHOS_ID']:
    if i not in list(phosphosite_df['PHOS_ID']):
        row = [{'GENE':'','PROTEIN':'','ACC_ID':'','HU_CHR_LOC':'','MOD_RSD':'','SITE_GRP_ID':0,'ORGANISM':'','MW_kD':0.00,'DOMAIN':'','SITE_+/-7_AA':'','LT_LIT':0.0,'MS_LIT':0.0,'MS_CST':0.0,'CST_CAT#':0.0,'PHOS_ID':i,'PHOS_ID2':'','PHOS_ID3':'','PHOS_ID4':''}]
        phosphosite_df = phosphosite_df.append(row, ignore_index = True)

Make a new kinase column in uppercase, with "_HUMAN" appended, to match the ID in the kinase table

In [20]:
# Take the IDs from the gene column

kinases = kin_sub_df.KIN_ACC_ID
kinases = list(kinases.drop_duplicates())

In [20]:
# Translate the IDs using Uniprot

# This takes around 15 minutes to run

# Create partial URL

url = "https://www.uniprot.org/uploadlists/"

# Define parameters

params = {
"from" : "ACC", # Kinase names are in format "ACC"
"to" : "ID", # Retrieve IDs in "ID" format
"format" : "tab", # Produce tab-delimited output
"query" : "", # The query protein ID will be defined during the loop
}

# Create an empty list to store the results in

results = []

for i in kinases: 
    params["query"] = str(i) # Enter the kinase ID
    data = urllib.parse.urlencode(params) 
    data = data.encode("utf-8")
    req = urllib.request.Request(url, data) # Run query in Uniprot
    with urllib.request.urlopen(req) as f:
       response = f.read()
    line = response.decode("utf-8") 
    results.append(line) # Store results

(If the cell above fails to run due to a connection error, un-comment the cell below and run it to use a snapshot of the data retrieved on 2020_01_13)

In [21]:
#results = ['From\tTo\nQ9BQI3\tE2AK1_HUMAN\n','From\tTo\nQ05655\tKPCD_HUMAN\n','From\tTo\nQ9P1W9\tPIM2_HUMAN\n','From\tTo\nQ9UQM7\tKCC2A_HUMAN\n','From\tTo\nP68400\tCSK21_HUMAN\n','From\tTo\nO14965\tAURKA_HUMAN\n','From\tTo\nP35269\tT2FA_HUMAN\n','From\tTo\nO95835\tLATS1_HUMAN\n','From\tTo\nP09619\tPGFRB_HUMAN\n','From\tTo\nP22455\tFGFR4_HUMAN\n','From\tTo\nP50613\tCDK7_HUMAN\n','From\tTo\nQ15208\tSTK38_HUMAN\n','From\tTo\nQ9UGJ0\tAAKG2_HUMAN\n','From\tTo\nQ13131\tAAPK1_HUMAN\n','From\tTo\nQ9NYV4\tCDK12_HUMAN\n','From\tTo\nQ16566\tKCC4_HUMAN\n','From\tTo\nP54646\tAAPK2_HUMAN\n','From\tTo\nQ9Y2H1\tST38L_HUMAN\n','From\tTo\nP49674\tKC1E_HUMAN\n','From\tTo\nQ14012\tKCC1A_HUMAN\n','From\tTo\nA9UF07\tA9UF07_HUMAN\n','From\tTo\nP07947\tYES_HUMAN\n','From\tTo\nP48736\tPK3CG_HUMAN\n','From\tTo\nQ9UHD2\tTBK1_HUMAN\n','From\tTo\nQ02763\tTIE2_HUMAN\n','From\tTo\nO94806\tKPCD3_HUMAN\n','From\tTo\nP07333\tCSF1R_HUMAN\n','From\tTo\nP05771-2\tKPCB_HUMAN\n','From\tTo\nP43405\tKSYK_HUMAN\n','From\tTo\nQ9UEW8\tSTK39_HUMAN\n','From\tTo\nP51617\tIRAK1_HUMAN\n','From\tTo\nO76039\tCDKL5_HUMAN\n','From\tTo\nQ15759\tMK11_HUMAN\n','From\tTo\nQ15303\tERBB4_HUMAN\n','From\tTo\nQ9BXM7\tPINK1_HUMAN\n','From\tTo\nP04049\tRAF1_HUMAN\n','From\tTo\nP15531\tNDKA_HUMAN\n','From\tTo\nP21127\tCD11B_HUMAN\n','From\tTo\nP49137\tMAPK2_HUMAN\n','From\tTo\nP27448-3\tMARK3_HUMAN\n','From\tTo\nP78368\tKC1G2_HUMAN\n','From\tTo\nQ04759\tKPCT_HUMAN\n','From\tTo\nP22694\tKAPCB_HUMAN\n','From\tTo\nQ9HCP0\tKC1G1_HUMAN\n','From\tTo\nQ13163\tMP2K5_HUMAN\n','From\tTo\nQ8N4C8\tMINK1_HUMAN\n','From\tTo\nO14757\tCHK1_HUMAN\n','From\tTo\nP17612\tKAPCA_HUMAN\n','From\tTo\nP27361\tMK03_HUMAN\n','From\tTo\nQ9Y572\tRIPK3_HUMAN\n','From\tTo\nQ92918\tM4K1_HUMAN\n','From\tTo\nP29323\tEPHB2_HUMAN\n', 'From\tTo\nQ8N5S9\tKKCC1_HUMAN\n', 'From\tTo\nQ9UQ88\tCD11A_HUMAN\n', 'From\tTo\nP48730\tKC1D_HUMAN\n', 'From\tTo\nQ9BUB5-2\tMKNK1_HUMAN\n', 'From\tTo\nP11021\tBIP_HUMAN\n', 'From\tTo\nQ16654\tPDK4_HUMAN\n', 'From\tTo\nO00444\tPLK4_HUMAN\n', 'From\tTo\nO14578\tCTRO_HUMAN\n', 'From\tTo\nQ9UQB9\tAURKC_HUMAN\n', 'From\tTo\nO14733\tMP2K7_HUMAN\n', 'From\tTo\nP42685\tFRK_HUMAN\n', 'From\tTo\nO00329\tPK3CD_HUMAN\n', 'From\tTo\nO75716\tSTK16_HUMAN\n', 'From\tTo\nQ00536\tCDK16_HUMAN\n', 'From\tTo\nP04626\tERBB2_HUMAN\n', 'From\tTo\nQ5VT25\tMRCKA_HUMAN\n', 'From\tTo\nO95747\tOXSR1_HUMAN\n', 'From\tTo\nP12931\tSRC_HUMAN\n', 'From\tTo\nP42338\tPK3CB_HUMAN\n', 'From\tTo\nQ00526\tCDK3_HUMAN\n', 'From\tTo\nQ9Y3S1\tWNK2_HUMAN\n', 'From\tTo\nQ16620\tNTRK2_HUMAN\n', 'From\tTo\nO75676\tKS6A4_HUMAN\n', 'From\tTo\nQ13418\tILK_HUMAN\n', 'From\tTo\nQ15569\tTESK1_HUMAN\n', 'From\tTo\nQ99986\tVRK1_HUMAN\n', 'From\tTo\nP41743\tKPCI_HUMAN\n', 'From\tTo\nP43403\tZAP70_HUMAN\n','From\tTo\nP11274\tBCR_HUMAN\n', 'From\tTo\nQ99640\tPMYT1_HUMAN\n', 'From\tTo\nQ8TDX7\tNEK7_HUMAN\n', 'From\tTo\nO14976\tGAK_HUMAN\n', 'From\tTo\nO43781\tDYRK3_HUMAN\n', 'From\tTo\nP42681\tTXK_HUMAN\n', 'From\tTo\nO00238\tBMR1B_HUMAN\n', 'From\tTo\nP29597\tTYK2_HUMAN\n', 'From\tTo\nQ14289\tFAK2_HUMAN\n', 'From\tTo\nQ9UBS0\tKS6B2_HUMAN\n', 'From\tTo\nP09769\tFGR_HUMAN\n', 'From\tTo\nQ13882\tPTK6_HUMAN\n', 'From\tTo\nO75385\tULK1_HUMAN\n', 'From\tTo\nP45984-2\tMK09_HUMAN\n', 'From\tTo\nP42680\tTEC_HUMAN\n', 'From\tTo\nQ9UKI8\tTLK1_HUMAN\n', 'From\tTo\nP06493\tCDK1_HUMAN\n', 'From\tTo\nQ13976-2\tKGP1_HUMAN\n', 'From\tTo\nQ8IVT5\tKSR1_HUMAN\n', 'From\tTo\nO96013\tPAK4_HUMAN\n', 'From\tTo\nQ9H1R3\tMYLK2_HUMAN\n', 'From\tTo\nO96017\tCHK2_HUMAN\n', 'From\tTo\nQ13557\tKCC2D_HUMAN\n', 'From\tTo\nP43250\tGRK6_HUMAN\n', 'From\tTo\nO00506\tSTK25_HUMAN\n', 'From\tTo\nP17948\tVGFR1_HUMAN\n', 'From\tTo\nQ9BYP7\tWNK3_HUMAN\n', 'From\tTo\nO43353\tRIPK2_HUMAN\n', 'From\tTo\nP08069\tIGF1R_HUMAN\n', 'From\tTo\nQ02779\tM3K10_HUMAN\n', 'From\tTo\nQ09013\tDMPK_HUMAN\n', 'From\tTo\nP37173\tTGFR2_HUMAN\n', 'From\tTo\nQ38SD2\tLRRK1_HUMAN\n', 'From\tTo\nP49759\tCLK1_HUMAN\n', 'From\tTo\nQ8IWQ3\tBRSK2_HUMAN\n', 'From\tTo\nQ15131\tCDK10_HUMAN\n', 'From\tTo\nP78527\tPRKDC_HUMAN\n', 'From\tTo\nQ9Y2I7\tFYV1_HUMAN\n', 'From\tTo\nO14730\tRIOK3_HUMAN\n', 'From\tTo\nQ12852\tM3K12_HUMAN\n', 'From\tTo\nQ16512\tPKN1_HUMAN\n', 'From\tTo\nQ96Q15\tSMG1_HUMAN\n', 'From\tTo\nO95382\tM3K6_HUMAN\n', 'From\tTo\nP53355\tDAPK1_HUMAN\n', 'From\tTo\nQ15139\tKPCD1_HUMAN\n', 'From\tTo\nO43318\tM3K7_HUMAN\n', 'From\tTo\nP10721\tKIT_HUMAN\n', 'From\tTo\nP19525\tE2AK2_HUMAN\n', 'From\tTo\nP29317\tEPHA2_HUMAN\n', 'From\tTo\nQ15831\tSTK11_HUMAN\n', 'From\tTo\nQ13557-8\tKCC2D_HUMAN\n', 'From\tTo\nQ9P0L2\tMARK1_HUMAN\n', 'From\tTo\nQ8WTQ7\tGRK7_HUMAN\n', 'From\tTo\nP46020\tKPB1_HUMAN\n', 'From\tTo\nQ6PHR2\tULK3_HUMAN\n', 'From\tTo\nQ16513\tPKN2_HUMAN\n', 'From\tTo\nQ7KZI7\tMARK2_HUMAN\n', 'From\tTo\nP49760\tCLK2_HUMAN\n', 'From\tTo\nQ9UIG0\tBAZ1B_HUMAN\n', 'From\tTo\nO00141\tSGK1_HUMAN\n', 'From\tTo\nP23458\tJAK1_HUMAN\n', 'From\tTo\nP21675\tTAF1_HUMAN\n', 'From\tTo\nQ96GX5\tGWL_HUMAN\n', 'From\tTo\nP00519\tABL1_HUMAN\n', 'From\tTo\nO00311\tCDC7_HUMAN\n', 'From\tTo\nP45983-2\tMK08_HUMAN\n', 'From\tTo\nP49841\tGSK3B_HUMAN\n', 'From\tTo\nQ16644\tMAPK3_HUMAN\n', 'From\tTo\nP51813\tBMX_HUMAN\n', 'From\tTo\nQ9NYY3\tPLK2_HUMAN\n', 'From\tTo\nP10398\tARAF_HUMAN\n', 'From\tTo\nQ9Y463\tDYR1B_HUMAN\n', 'From\tTo\nQ9HBH9\tMKNK2_HUMAN\n', 'From\tTo\nQ86V86\tPIM3_HUMAN\n', 'From\tTo\nP06239\tLCK_HUMAN\n', 'From\tTo\nQ04912\tRON_HUMAN\n', 'From\tTo\nQ15300\tQ15300_HUMAN\n', 'From\tTo\nQ8TAS1\tUHMK1_HUMAN\n', 'From\tTo\nQ14164\tIKKE_HUMAN\n', 'From\tTo\nP53778\tMK12_HUMAN\n', 'From\tTo\nP45985\tMP2K4_HUMAN\n', 'From\tTo\nQ9HAZ1\tCLK4_HUMAN\n', 'From\tTo\nO15530\tPDPK1_HUMAN\n', 'From\tTo\nQ99683\tM3K5_HUMAN\n', 'From\tTo\nQ96NX5\tKCC1G_HUMAN\n', 'From\tTo\nP35968\tVGFR2_HUMAN\n', 'From\tTo\nP34947\tGRK5_HUMAN\n', 'From\tTo\nP19784\tCSK22_HUMAN\n', 'From\tTo\nP37023\tACVL1_HUMAN\n', 'From\tTo\nQ5S007\tLRRK2_HUMAN\n', 'From\tTo\nP57059\tSIK1_HUMAN\n', 'From\tTo\nQ16539\tMK14_HUMAN\n', 'From\tTo\nQ2M2I8\tAAK1_HUMAN\n', 'From\tTo\nP07332\tFES_HUMAN\n', 'From\tTo\nQ96J92\tWNK4_HUMAN\n', 'From\tTo\nO15111\tIKKA_HUMAN\n', 'From\tTo\nP24941\tCDK2_HUMAN\n', 'From\tTo\nQ9UBE8\tNLK_HUMAN\n', 'From\tTo\nP21980\tTGM2_HUMAN\n', 'From\tTo\nO14638\tENPP3_HUMAN\n', 'From\tTo\nO00418\tEF2K_HUMAN\n', 'From\tTo\nQ13976\tKGP1_HUMAN\n', 'From\tTo\nP28482\tMK01_HUMAN\n', 'From\tTo\nO94921\tCDK14_HUMAN\n', 'From\tTo\nP49841-2\tGSK3B_HUMAN\n', 'From\tTo\nP49840\tGSK3A_HUMAN\n', 'From\tTo\nQ15119\tPDK2_HUMAN\n', 'From\tTo\nQ6SA08\tTSSK4_HUMAN\n', 'From\tTo\nP17612-2\tKAPCA_HUMAN\n', 'From\tTo\nP53671\tLIMK2_HUMAN\n', 'From\tTo\nP08581\tMET_HUMAN\n', 'From\tTo\nQ13627\tDYR1A_HUMAN\n', 'From\tTo\nQ86Z02\tHIPK1_HUMAN\n', 'From\tTo\nP51956\tNEK3_HUMAN\n', 'From\tTo\nQ8NG66\tNEK11_HUMAN\n', 'From\tTo\nP00558\tPGK1_HUMAN\n', 'From\tTo\nO60674\tJAK2_HUMAN\n', 'From\tTo\nP11802\tCDK4_HUMAN\n', 'From\tTo\nQ99759\tM3K3_HUMAN\n', 'From\tTo\nP36897\tTGFR1_HUMAN\n', 'From\tTo\nQ8IW41\tMAPK5_HUMAN\n', 'From\tTo\nQ9UIK4\tDAPK2_HUMAN\n', 'From\tTo\nP24723\tKPCL_HUMAN\n', 'From\tTo\nP53350\tPLK1_HUMAN\n', 'From\tTo\nQ9BWU1\tCDK19_HUMAN\n', 'From\tTo\nQ00535\tCDK5_HUMAN\n', 'From\tTo\nP22392\tNDKB_HUMAN\n', 'From\tTo\nQ6P2M8\tKCC1B_HUMAN\n', 'From\tTo\nQ9BZL6\tKPCD2_HUMAN\n', 'From\tTo\nQ13554\tKCC2B_HUMAN\n', 'From\tTo\nQ9P286\tPAK5_HUMAN\n', 'From\tTo\nP36896\tACV1B_HUMAN\n', 'From\tTo\nO14936\tCSKP_HUMAN\n', 'From\tTo\nP22607\tFGFR3_HUMAN\n', 'From\tTo\nQ86UE8\tTLK2_HUMAN\n', 'From\tTo\nP52564\tMP2K6_HUMAN\n', 'From\tTo\nQ9P2K8\tE2AK4_HUMAN\n', 'From\tTo\nP42679\tMATK_HUMAN\n', 'From\tTo\nQ86Y07\tVRK2_HUMAN\n', 'From\tTo\nQ13464\tROCK1_HUMAN\n', 'From\tTo\nQ9Y5P4\tCERT_HUMAN\n', 'From\tTo\nQ13237\tKGP2_HUMAN\n', 'From\tTo\nP16591\tFER_HUMAN\n', 'From\tTo\nP50750\tCDK9_HUMAN\n', 'From\tTo\nQ02750\tMP2K1_HUMAN\n', 'From\tTo\nQ14680\tMELK_HUMAN\n', 'From\tTo\nP32298\tGRK4_HUMAN\n', 'From\tTo\nQ9H0K1\tSIK2_HUMAN\n', 'From\tTo\nP46734\tMP2K3_HUMAN\n', 'From\tTo\nQ00534\tCDK6_HUMAN\n', 'From\tTo\nP49336\tCDK8_HUMAN\n', 'From\tTo\nQ05397\tFAK1_HUMAN\n', 'From\tTo\nP54753\tEPHB3_HUMAN\n', 'From\tTo\nQ06418\tTYRO3_HUMAN\n', 'From\tTo\nQ15418\tKS6A1_HUMAN\n', 'From\tTo\nP16234\tPGFRA_HUMAN\n', 'From\tTo\nP25098\tARBK1_HUMAN\n', 'From\tTo\n', 'From\tTo\nQ9Y2U5\tM3K2_HUMAN\n', 'From\tTo\nQ9Y478\tAAKB1_HUMAN\n', 'From\tTo\nP36507\tMP2K2_HUMAN\n', 'From\tTo\nP52333\tJAK3_HUMAN\n', 'From\tTo\nQ8IWU2\tLMTK2_HUMAN\n', 'From\tTo\nQ08881\tITK_HUMAN\n', 'From\tTo\nP41240\tCSK_HUMAN\n', 'From\tTo\nQ13164\tMK07_HUMAN\n', 'From\tTo\nQ9H2X6\tHIPK2_HUMAN\n', 'From\tTo\nQ9UL54\tTAOK2_HUMAN\n', 'From\tTo\nP42345\tMTOR_HUMAN\n', 'From\tTo\nQ13523\tPRP4B_HUMAN\n', 'From\tTo\nP42336\tPK3CA_HUMAN\n', 'From\tTo\nQ9HC98\tNEK6_HUMAN\n', 'From\tTo\nP48729\tKC1A_HUMAN\n', 'From\tTo\nQ13546\tRIPK1_HUMAN\n', 'From\tTo\nQ9UPZ9\tICK_HUMAN\n', 'From\tTo\nQ9Y6E0\tSTK24_HUMAN\n', 'From\tTo\nQ92630\tDYRK2_HUMAN\n', 'From\tTo\nO43683\tBUB1_HUMAN\n', 'From\tTo\nQ02156\tKPCE_HUMAN\n', 'From\tTo\nQ13188\tSTK3_HUMAN\n', 'From\tTo\nP14618\tKPYM_HUMAN\n', 'From\tTo\nP53004\tBIEA_HUMAN\n', 'From\tTo\nQ13535\tATR_HUMAN\n', 'From\tTo\nO94804\tSTK10_HUMAN\n', 'From\tTo\nP42684\tABL2_HUMAN\n', 'From\tTo\nP53667\tLIMK1_HUMAN\n', 'From\tTo\nP08631\tHCK_HUMAN\n', 'From\tTo\nQ16584\tM3K11_HUMAN\n','From\tTo\nQ15120\tPDK3_HUMAN\n','From\tTo\nO75116\tROCK2_HUMAN\n','From\tTo\nQ99558\tM3K14_HUMAN\n','From\tTo\nP23443\tKS6B1_HUMAN\n','From\tTo\nQ96QT4\tTRPM7_HUMAN\n','From\tTo\nQ12866\tMERTK_HUMAN\n','From\tTo\nQ9NQU5\tPAK6_HUMAN\n','From\tTo\nQ13153\tPAK1_HUMAN\n','From\tTo\nQ9BX84\tTRPM6_HUMAN\n','From\tTo\nQ9H4A3\tWNK1_HUMAN\n','From\tTo\nP06213\tINSR_HUMAN\n','From\tTo\nP41279\tM3K8_HUMAN\n','From\tTo\nQ9H093\tNUAK2_HUMAN\n','From\tTo\nP45984\tMK09_HUMAN\n','From\tTo\nQ9NRM7\tLATS2_HUMAN\n','From\tTo\nQ96RR4\tKKCC2_HUMAN\n','From\tTo\nO95819\tM4K4_HUMAN\n','From\tTo\nP67870\tCSK2B_HUMAN\n','From\tTo\nP11362\tFGFR1_HUMAN\n','From\tTo\nO75460\tERN1_HUMAN\n','From\tTo\nP35626\tARBK2_HUMAN\n','From\tTo\nQ96BR1\tSGK3_HUMAN\n','From\tTo\nQ13315\tATM_HUMAN\n','From\tTo\nP04629\tNTRK1_HUMAN\n','From\tTo\nQ6P5Z2\tPKN3_HUMAN\n','From\tTo\nO15264\tMK13_HUMAN\n','From\tTo\nQ8NEV4\tMYO3A_HUMAN\n','From\tTo\nP06241\tFYN_HUMAN\n','From\tTo\nP33981\tTTK_HUMAN\n','From\tTo\nQ15835\tGRK1_HUMAN\n','From\tTo\nQ8N752\tKC1AL_HUMAN\n','From\tTo\nP17252\tKPCA_HUMAN\n','From\tTo\nQ12851\tM4K2_HUMAN\n','From\tTo\nQ9P289\tSTK26_HUMAN\n','From\tTo\nQ9H3Y6\tSRMS_HUMAN\n','From\tTo\nQ8TDC3\tBRSK1_HUMAN\n','From\tTo\nQ8TF76\tHASP_HUMAN\n','From\tTo\nO14920\tIKKB_HUMAN\n','From\tTo\nQ9BUB5\tMKNK1_HUMAN\n','From\tTo\nQ96GD4\tAURKB_HUMAN\n','From\tTo\nQ15349\tKS6A2_HUMAN\n','From\tTo\nP51451\tBLK_HUMAN\n','From\tTo\nO60885\tBRD4_HUMAN\n','From\tTo\nP51955\tNEK2_HUMAN\n','From\tTo\nP11309\tPIM1_HUMAN\n','From\tTo\nQ13043\tSTK4_HUMAN\n','From\tTo\nP00540\tMOS_HUMAN\n','From\tTo\nQ8IVH8\tM4K3_HUMAN\n','From\tTo\nQ8IXL6\tFA20C_HUMAN\n','From\tTo\nP54764\tEPHA4_HUMAN\n','From\tTo\nP78362\tSRPK2_HUMAN\n','From\tTo\nQ9UQ88-10\tCD11A_HUMAN\n','From\tTo\nP51817\tPRKX_HUMAN\n','From\tTo\nQ16659\tMK06_HUMAN\n','From\tTo\nQ13555\tKCC2G_HUMAN\n','From\tTo\nQ15118\tPDK1_HUMAN\n','From\tTo\nP45983\tMK08_HUMAN\n','From\tTo\nP27448\tMARK3_HUMAN\n','From\tTo\nP31751\tAKT2_HUMAN\n','From\tTo\nQ9H422\tHIPK3_HUMAN\n','From\tTo\nP05771\tKPCB_HUMAN\n','From\tTo\nQ96RG2\tPASK_HUMAN\n','From\tTo\nQ96KB5\tTOPK_HUMAN\n','From\tTo\nQ9NZJ5\tE2AK3_HUMAN\n','From\tTo\nP15056\tBRAF_HUMAN\n','From\tTo\nQ8TD19\tNEK9_HUMAN\n','From\tTo\nQ15746\tMYLK_HUMAN\n','From\tTo\nP05129\tKPCG_HUMAN\n','From\tTo\nQ8IV63\tVRK3_HUMAN\n','From\tTo\nP36888\tFLT3_HUMAN\n','From\tTo\nQ96SB4\tSRPK1_HUMAN\n','From\tTo\nQ05513\tKPCZ_HUMAN\n','From\tTo\nP30291\tWEE1_HUMAN\n','From\tTo\nQ9H4B4\tPLK3_HUMAN\n','From\tTo\nQ14004\tCDK13_HUMAN\n','From\tTo\nP53779\tMK10_HUMAN\n','From\tTo\nQ96PY6\tNEK1_HUMAN\n','From\tTo\nQ8IZL9\tCDK20_HUMAN\n','From\tTo\nP30530\tUFO_HUMAN\n','From\tTo\nO43293\tDAPK3_HUMAN\n','From\tTo\nP21802\tFGFR2_HUMAN\n','From\tTo\nQ8TDC3-2\tBRSK1_HUMAN\n','From\tTo\nQ07912\tACK1_HUMAN\n','From\tTo\nQ9Y243\tAKT3_HUMAN\n','From\tTo\nQ13233\tM3K1_HUMAN\n','From\tTo\nQ96S44\tPRPK_HUMAN\n','From\tTo\nQ9UM73\tALK_HUMAN\n','From\tTo\nO75582\tKS6A5_HUMAN\n','From\tTo\nP00533\tEGFR_HUMAN\n','From\tTo\nQ13177\tPAK2_HUMAN\n','From\tTo\nQ9Y4K4\tM4K5_HUMAN\n','From\tTo\nQ9UEE5\tST17A_HUMAN\n','From\tTo\nQ8TD08\tMK15_HUMAN\n','From\tTo\nP07949\tRET_HUMAN\n','From\tTo\nP31749\tAKT1_HUMAN\n','From\tTo\nP07948\tLYN_HUMAN\n','From\tTo\nP51812\tKS6A3_HUMAN\n','From\tTo\nQ6IQ55\tTTBK2_HUMAN\n','From\tTo\nQ06187\tBTK_HUMAN\n','From\tTo\nQ9NWZ3\tIRAK4_HUMAN\n','From\tTo\nQ9UKE5\tTNIK_HUMAN\n','From\tTo\nQ5TCY1\tTTBK1_HUMAN\n','From\tTo\nQ9NYL2\tM3K20_HUMAN\n','From\tTo\nQ9Y6R4\tM3K4_HUMAN\n','From\tTo\nO60285\tNUAK1_HUMAN\n','From\tTo\nQ8IYT8\tULK2_HUMAN\n']

In [22]:
# Split up the search results into a list of lists

results2=[]

for n,i in enumerate(results):
    results2.append(i.split())

In [23]:
# Make a dictionary of proteins

proteindict={}

for i in results2: # i = one kinase and all of its possible translations
    if(len(i)) > 2: # ignore empty lists
        humans = 0
        for e in range(3,len(i),2): # For every other item in the list (i.e. a potentially correct ID)
            if "_HUMAN" in str(i[e]): # check how many "human" options there are
                 humans += 1 
        if humans > 1: # If there are multiple "human" options
            for j in range(3,len(i),2):
                if str(i[j]) == str(i[2]) + "_HUMAN": # keep the one that equates to the original ID with
                    # suffix "_HUMAN"
                    proteindict[str(i[2])] = str(i[j])
        if humans == 1: # If there is only one "human" option, choose it
            for h in range(3,len(i),2):
                if "_HUMAN" in str(i[h]):
                    proteindict[str(i[2])] = str(i[h])

In [24]:
# Translate the kinases in the data frame and insert into new column

data = []

for n,i in enumerate(kin_sub_df.KIN_ACC_ID): # Iterate over original data frame's IDs
    if i in proteindict.keys():
        kinase = proteindict.get(i)
        data.append(kinase)
    else:
        data.append(str(i) + "_NOT_FOUND_IN_UNIPROT")

data = pd.Series(data)
kin_sub_df = kin_sub_df.assign(HUMAN_KINASE = data)

Make GENE entries uppercase

In [25]:
# This takes a minute to run

uppercase_kinase = []

for n,i in enumerate(phosphosite_df.iterrows()):
    uppercase_kinase.append(phosphosite_df.GENE[n].upper())

uppercase_kinase = pd.Series(uppercase_kinase)

phosphosite_df = phosphosite_df.assign(GENE = uppercase_kinase)

In [26]:
uppercase_kinase = []

for n,i in enumerate(kin_sub_df.iterrows()):
    uppercase_kinase.append(kin_sub_df.GENE[n].upper())

uppercase_kinase = pd.Series(uppercase_kinase)

kin_sub_df = kin_sub_df.assign(GENE = uppercase_kinase)

Remove -p from MOD_RSD in phosphosite_df

In [27]:
# This takes a minute to run

mod_rsd = []

for n,i in enumerate(phosphosite_df.iterrows()):
    if phosphosite_df.MOD_RSD[n][-2:] == "-p":
        mod_rsd.append(phosphosite_df.MOD_RSD[n][0:-2])
    else:
        mod_rsd.append(phosphosite_df.MOD_RSD[n])

mod_rsd = pd.Series(mod_rsd)

phosphosite_df = phosphosite_df.assign(MOD_RSD = mod_rsd)

Drop any unnecessary columns

In [28]:
# ORGANISM is not required as the web app will be human-specific
# and we have already ensured that only human targets are included

phosphosite_df = phosphosite_df.drop(['ORGANISM'], axis=1)
phosphosite_df = phosphosite_df.reset_index(drop = True)

In [29]:
# KIN_ACC_ID and SUB_ACC_ID will be available in kinase table
# KINASE, KIN_ORGANISM, SUBSTRATE, SUB_ORGANISM, SUB_GENE_ID, 
# SUB_MOD_RSD are not required
# SUB_GENE will be in phosphosite_df as GENE
# SITE_GRP_ID, SITE_+/-7_AA and DOMAIN will be available in phosphosite_df

kin_sub_df = kin_sub_df.drop(['KIN_ACC_ID', 'SUB_ACC_ID', 'KINASE',
                              'KIN_ORGANISM', 'SUBSTRATE', 'SUB_ORGANISM', 
                              'SUB_GENE_ID', 'SUB_MOD_RSD',
                              'SUB_GENE', 'SITE_GRP_ID', 'SITE_+/-7_AA',
                              'DOMAIN'], axis=1)
kin_sub_df = kin_sub_df.reset_index(drop = True)

In [30]:
# PROTEIN, ACC_ID and GENE_ID will be available in the kinase table
# ORGANISM is not required
# GENE, HU_CHR_LOC, MW_kD, SITE_GRP_ID, MOD_RSD, DOMAIN, and SITE_+/-7_AA
# will be available in phosphosite_df

disease_df = disease_df.drop(['PROTEIN', 'ACC_ID', 'GENE_ID',  
                              'ORGANISM', 'GENE', 'HU_CHR_LOC', 'MW_kD',
                              'SITE_GRP_ID', 'MOD_RSD', 'DOMAIN', 
                               'SITE_+/-7_AA'], axis=1)
disease_df = disease_df.reset_index(drop = True)

Add a column ISOFORM to the phosphosite_df table to indicate whether the row is for an isoform (1) or not (0)

In [31]:
yes_no_isoform = []

for n,i in enumerate(phosphosite_df.ACC_ID):
    if "-" in i: # In a UniProt ID, "-" signifies an isoform
        yes_no_isoform.append(1)
    elif "iso" in phosphosite_df.PROTEIN[n]: # For some records, the presence of "iso" 
        # in this field is the only way the isoform is indicated
        yes_no_isoform.append(1)
    else:
        yes_no_isoform.append(0)

yes_no_isoform = pd.Series(yes_no_isoform)

phosphosite_df = phosphosite_df.assign(ISOFORM = yes_no_isoform)

Remove any duplicated rows (excluding isoforms)

In [32]:
uniq = []
count = 0

for n,i in enumerate(phosphosite_df.PHOS_ID):
    if phosphosite_df.ISOFORM[n] == 0: 
        id_ = i
    else:
        count += 1
        id_ = str(i)+str(count)
    uniq.append(id_)

phosphosite_df = phosphosite_df.assign(uniq = uniq)
phosphosite_df = phosphosite_df.drop_duplicates(subset = 'uniq', keep = 'first')
phosphosite_df = phosphosite_df.drop(['uniq'], axis=1)
phosphosite_df = phosphosite_df.reset_index(drop = True)

Add primary key columns

In [33]:
prim_key = []

count = 1

for i in phosphosite_df.PHOS_ID:
    key = "PH"+"{:07d}".format(count)
    prim_key.append(key)
    count += 1

prim_key = pd.Series(prim_key)

phosphosite_df = phosphosite_df.assign(ID_PH = prim_key)

In [34]:
prim_key = []

count = 1

for i in kin_sub_df.PHOS_ID:
    key = "KS"+"{:07d}".format(count)
    prim_key.append(key)
    count += 1

prim_key = pd.Series(prim_key)

kin_sub_df = kin_sub_df.assign(ID_KS = prim_key)

In [35]:
prim_key = []

count = 1

for i in disease_df.PHOS_ID:
    key = "DP"+"{:07d}".format(count)
    prim_key.append(key)
    count += 1

prim_key = pd.Series(prim_key)

disease_df = disease_df.assign(ID_DP = prim_key)

Make csv files

In [36]:
phosphosite_df.to_csv("phosphosites.csv", index = False)

In [37]:
kin_sub_df.to_csv("kinases_phosphosites.csv", index = False)

In [38]:
disease_df.to_csv("phosphosites_diseases.csv", index = False)